author: Chunhui Li 

email: lchwestlife@gmail.com 

An walkthrougth exmaple jupyter notebook 

In [8]:
from bond_valence_processor import BondValenceProcessor
from pymatgen.core import Structure
from BVparams_search import TheoreticalBondValenceSolver, BVParamSolver

In [7]:
# import cif from folder using pymatgen 
aspod = Structure.from_file("1011191_aspod.cif")

In [9]:
TheoreticalBondValenceSolver()

FileNotFoundError: [Errno 2] No such file or directory: 'params/dict_matID_possible_species.json'

In [4]:
cations = ['Na'] # a list of cation species 
my_api_key = "YgzOEXsODWlsR0J9P5aSjX2CxHuZX9Zv"
# algos = ['shgo', 'brute', 'diff', 'dual_annealing', 'direct']
algos = ['shgo']
processor = BondValenceProcessor(my_api_key, algos, cations)
    
for cation in cations:
    processor.process_cation_system(cation)



Processing Na-O system...


Retrieving SummaryDoc documents:   0%|          | 0/5119 [00:00<?, ?it/s]

Getting possible species: 100%|██████████| 5119/5119 [00:00<00:00, 1126535.61it/s]


Retrieving BondingDoc documents:   0%|          | 0/4984 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from pymatgen.core import Element

# Get all elements from periodic table
all_elements = [el.symbol for el in Element]
print("All elements in periodic table:")
print(all_elements)


In [29]:
from mp_api.client import MPRester
with MPRester(api_key=my_api_key) as mpr:
    docs = mpr.materials.summary.search(elements=['Xe', 'O'],
                energy_above_hull=(0.000, 0.05),
                fields=['material_id', 'structure', 'possible_species'])

Retrieving SummaryDoc documents:   0%|          | 0/11 [00:00<?, ?it/s]

In [30]:
docs

[MPDataDoc<SummaryDoc>(
 material_id=MPID(mp-1198484),
 structure=Structure Summary
 Lattice
     abc : 6.357991 10.915356 16.461166
  angles : 90.0 90.0 90.0
  volume : 1142.4005616444965
       A : 6.357991 0.0 0.0
       B : 0.0 10.915356 0.0
       C : 0.0 0.0 16.461166
     pbc : True True True
 PeriodicSite: Na (5.968, 9.199, 8.616) [0.9387, 0.8427, 0.5234]
 PeriodicSite: Na (0.3895, 7.174, 0.3853) [0.06127, 0.6573, 0.02341]
 PeriodicSite: Na (3.569, 3.741, 7.845) [0.5613, 0.3427, 0.4766]
 PeriodicSite: Na (2.789, 1.717, 16.08) [0.4387, 0.1573, 0.9766]
 PeriodicSite: Na (0.3895, 1.717, 7.845) [0.06127, 0.1573, 0.4766]
 PeriodicSite: Na (5.968, 3.741, 16.08) [0.9387, 0.3427, 0.9766]
 PeriodicSite: Na (2.789, 7.174, 8.616) [0.4387, 0.6573, 0.5234]
 PeriodicSite: Na (3.569, 9.199, 0.3853) [0.5613, 0.8427, 0.02341]
 PeriodicSite: Na (0.737, 9.449, 12.04) [0.1159, 0.8657, 0.7317]
 PeriodicSite: Na (5.621, 6.924, 3.813) [0.8841, 0.6343, 0.2317]
 PeriodicSite: Na (2.442, 3.992, 4.417) [

In [ ]:
def get_possible_species(self, save_dir: str, docs: List[MaterialData]) -> List[str]:
        """Extract possible species from materials documents"""
        species_data = {
            doc.material_id: doc.possible_species
            for doc in tqdm(docs, desc='Getting possible species')
            if doc.possible_species
        }
        
        output_file = Path(save_dir) / "params" / "dict_matID_possible_species.json"
        with output_file.open('w') as f:
            json.dump(species_data, f)
            
        return list(species_data.keys())

In [19]:
list([e.material_id for e in docs])

[MPID(mp-1198484),
 MPID(mp-1191299),
 MPID(mp-1194972),
 MPID(mp-557213),
 MPID(mp-23568),
 MPID(mp-1204146),
 MPID(mp-560359),
 MPID(mp-581488),
 MPID(mp-556244),
 MPID(mp-1199259),
 MPID(mp-560077)]

In [26]:
bond_docs = mpr.materials.bonds.search(material_ids=['mp-560359'], 
                                       fields=['material_id', 'structure_graph', 'formula_pretty'])

Retrieving BondingDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# bond_valence_processor.py
from tqdm import tqdm
import os
import numpy as np
import json
from mp_api.client import MPRester
from BVparams_search import TheoreticalBondValenceSolver
from BVparams_search import BVParamSolver

class BondValenceProcessor:
    def __init__(self, api_key, algos):
        self.api_key = api_key
        self.algos = algos
        
    def process_cation_system(self, cation):
        """Process a single cation-O system"""
        print(f'start calculating {cation}-O system:')
        
        # Download dataset
        docs = self.download_materials_data(cation)
        
        # Setup directories and get material IDs
        res_dir = f'res/{cation}O'
        mids = self.get_possible_species_perID(res_dir, docs)
        
        # Get bonding data
        bonds_docs = self.download_bonding_data(mids)
        
        # Initialize data structures
        dict_sij_perMatID = {}
        dict_charges_perMatID = {}
        
        # Handle previously solved cases
        solved_materID, no_solu = self.get_previous_results(res_dir)
        
        # Initialize solver
        sij_solver = TheoreticalBondValenceSolver(
            species_matID_path=f'{res_dir}/params/dict_matID_possible_species.json'
        )
        
        # Process each material
        self.process_materials(bonds_docs, solved_materID, sij_solver, dict_sij_perMatID, 
                         dict_charges_perMatID, no_solu, res_dir, cation)
        
        # Save results
        self.save_results(res_dir, dict_sij_perMatID, dict_charges_perMatID)

    def download_materials_data(self, cation):
        """Download materials data from Materials Project"""
        with MPRester(api_key=self.api_key) as mpr:
            return mpr.materials.summary.search(
                elements=[cation, 'O'],
                energy_above_hull=(0.000, 0.05),
                fields=['material_id', 'possible_species']
            )

    def download_bonding_data(self, mids):
        """Download bonding data from Materials Project"""
        with MPRester(api_key=self.api_key) as mpr:
            return mpr.materials.bonds.search(
                material_ids=mids,
                fields=['material_id', 'structure_graph', 'formula_pretty']
            )

    def get_previous_results(self, res_dir):
        """Get previously solved cases"""
        if os.path.exists(f'{res_dir}/R0Bs/shgo'):
            solved_sol = os.listdir(f'{res_dir}/R0Bs/shgo/')
            solved_materID = [e.split('.txt')[0] for e in solved_sol]
            if '.ipynb_checkpoints' in solved_materID:
                solved_materID.remove('.ipynb_checkpoints')
            
            no_solu = np.loadtxt(f'{res_dir}/no_solu/shgo.txt', dtype=str).tolist()
            nosol_IDs = [e[0] for e in no_solu]
            nosol_IDs = list(set(nosol_IDs))
            solved_materID += nosol_IDs
        else:
            solved_materID = []
            no_solu = []
        
        return solved_materID, no_solu

    def process_materials(self, bonds_docs, solved_materID, sij_solver, dict_sij_perMatID,
                         dict_charges_perMatID, no_solu, res_dir, cation):
        """Process each material in the dataset"""
        for Li_mater in tqdm(bonds_docs):
            matID = Li_mater.material_id
            reduced_formula = Li_mater.formula_pretty
            cur_struct = Li_mater.structure_graph.structure
            
            # Get Sij values
            cur_network_valence_dict, cur_bond_type_list, cur_bondL_dict, dict_charge = sij_solver.get_sij(
                matID, cur_struct, Li_mater.structure_graph
            )
            
            dict_sij_perMatID[matID] = cur_network_valence_dict
            dict_charges_perMatID[matID] = dict_charge

            if matID in solved_materID:
                continue
                
            self.process_algorithm_results(cur_network_valence_dict, no_solu, res_dir,
                                    cation, matID, reduced_formula, cur_bond_type_list,
                                    cur_bondL_dict)

    def process_algorithm_results(self, cur_network_valence_dict, no_solu, res_dir,
                                cation, matID, reduced_formula, cur_bond_type_list,
                                cur_bondL_dict):
        """Process results using different algorithms"""
        for alg in self.algos:
            if not cur_network_valence_dict:
                no_solu.append((matID, cation, 'O', reduced_formula, 'no_network_sol'))
                np.savetxt(f'{res_dir}/no_solu/{alg}.txt', no_solu, fmt='%s')
                continue
                
            bv_solver = BVParamSolver(save_dir=res_dir, algo=alg, no_sol=no_solu)
            
            new_R0_B_LiO = bv_solver.solve_R0Bs(
                cation=cation,
                anion='O',
                bond_type_list=cur_bond_type_list,
                networkValence_dict=cur_network_valence_dict,
                bondLen_dict=cur_bondL_dict,
                materID=matID,
                chem_formula=reduced_formula,
                R0_bounds=(0, 5),
            )
            
            if new_R0_B_LiO:
                np.savetxt(f'{res_dir}/R0Bs/{alg}/{matID}.txt', new_R0_B_LiO)

    def save_results(self, res_dir, dict_sij_perMatID, dict_charges_perMatID):
        """Save final results"""
        with open(f'{res_dir}/dict_sijs.json', 'w') as fopen:
            json.dump(dict_sij_perMatID, fopen)
        
        with open(f'{res_dir}/dict_charges.json', 'w') as fopen:
            json.dump(dict_charges_perMatID, fopen)

# Main processing
if __name__ == "__main__":
    my_api_key = "YgzOEXsODWlsR0J9P5aSjX2CxHuZX9Zv"
    algos = ['shgo', 'brute', 'diff', 'dual_annealing', 'direct']
    processor = BondValenceProcessor(my_api_key, algos)
    
    for cation in cations:
        processor.process_cation_system(cation)

start calculating Ti-O system:


Retrieving SummaryDoc documents:   0%|          | 0/2284 [00:00<?, ?it/s]

Retrieving BondingDoc documents:   0%|          | 0/2235 [00:00<?, ?it/s]

 31%|███       | 685/2235 [1:01:49<4:29:04, 10.42s/it] /global/home/users/chli/.local/lib/python3.9/site-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
 55%|█████▌    | 1237/2235 [3:46:27<10:49,  1.54it/s]    spglib: Attempt 0 tolerance = 1.000000e-02 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 1 tolerance = 9.500000e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 2 tolerance = 9.025000e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 3 tolerance = 8.573750e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 4 tolerance = 8.145062e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 5 tolerance = 7.737809e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 6 tolerance = 7.350919e-03 failed(line 800, /project/src/spacegroup.c).
spglib: Attempt 7 tolerance =